In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score
import pickle
from collections import defaultdict
from sklearn.utils import shuffle
import os
import cv2
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import pandas as pd

from torchinfo import summary
from sklearn.metrics import confusion_matrix
import seaborn as sns



In [2]:
import numpy as np
import os

# Define the folder path
folder_path = r"D:\PYTHONIG\newwindow\NOTEBOOKS_2025\aprilmay2025\data\numpy\optunadata\mel"

# Load the numpy files into the respective arrays with the correct capitalized naming
eeg_fold_1 = np.load(os.path.join(folder_path, 'MEL_DATA_FOLD_fold_1.npy'))
labels_fold_1 = np.load(os.path.join(folder_path, 'MEL_LABELS_FOLD_fold_1.npy'))
patients_fold_1 = np.load(os.path.join(folder_path, 'MEL_PATIENTS_FOLD_fold_1.npy'))

eeg_fold_2 = np.load(os.path.join(folder_path, 'MEL_DATA_FOLD_fold_2.npy'))
labels_fold_2 = np.load(os.path.join(folder_path, 'MEL_LABELS_FOLD_fold_2.npy'))
patients_fold_2 = np.load(os.path.join(folder_path, 'MEL_PATIENTS_FOLD_fold_2.npy'))

eeg_fold_3 = np.load(os.path.join(folder_path, 'MEL_DATA_FOLD_fold_3.npy'))
labels_fold_3 = np.load(os.path.join(folder_path, 'MEL_LABELS_FOLD_fold_3.npy'))
patients_fold_3 = np.load(os.path.join(folder_path, 'MEL_PATIENTS_FOLD_fold_3.npy'))

eeg_fold_4 = np.load(os.path.join(folder_path, 'MEL_DATA_FOLD_fold_4.npy'))
labels_fold_4 = np.load(os.path.join(folder_path, 'MEL_LABELS_FOLD_fold_4.npy'))
patients_fold_4 = np.load(os.path.join(folder_path, 'MEL_PATIENTS_FOLD_fold_4.npy'))

eeg_fold_5 = np.load(os.path.join(folder_path, 'MEL_DATA_FOLD_fold_5.npy'))
labels_fold_5 = np.load(os.path.join(folder_path, 'MEL_LABELS_FOLD_fold_5.npy'))
patients_fold_5 = np.load(os.path.join(folder_path, 'MEL_PATIENTS_FOLD_fold_5.npy'))

eeg_folds = [eeg_fold_1, eeg_fold_2, eeg_fold_3, eeg_fold_4, eeg_fold_5]
labels_folds = [labels_fold_1, labels_fold_2, labels_fold_3, labels_fold_4, labels_fold_5]
patients_folds = [patients_fold_1, patients_fold_2, patients_fold_3, patients_fold_4, patients_fold_5]

for i in range(len(eeg_folds)):
    eeg_folds[i] = eeg_folds[i].astype(np.float16)

In [3]:
def data_balancer(data, labels, factor):
    # Count the number of samples in each class
    num_class_0 = np.sum(labels == 0)
    num_class_1 = np.sum(labels == 1)
    num_class_2 = np.sum(labels == 2)

    # Find the minimum number of samples across all classes
    min_samples = min(num_class_0, num_class_1, num_class_2)

    # Calculate the number of samples to take from each class
    samples_per_class = min_samples // factor

    # Randomly sample 'samples_per_class' from each class
    class_0_indices = np.random.choice(np.where(labels == 0)[0], samples_per_class, replace=False)
    class_1_indices = np.random.choice(np.where(labels == 1)[0], samples_per_class, replace=False)
    class_2_indices = np.random.choice(np.where(labels == 2)[0], samples_per_class, replace=False)

    # Combine balanced indices
    balanced_indices = np.concatenate((class_0_indices, class_1_indices, class_2_indices))

    # Shuffle the balanced indices
    np.random.shuffle(balanced_indices)

    # Create balanced training data and labels
    balanced_data = data[balanced_indices]
    balanced_labels = labels[balanced_indices]

    return balanced_data, balanced_labels

In [4]:
class EarlyStopping:
    def __init__(self, patience=5):
        """
        Initializes the early stopping mechanism based on divergence detection.

        Args:
            patience (int): Number of consecutive epochs with increasing validation loss
                            before stopping.
        """
        self.patience = patience
        self.best_loss = None
        self.counter = 0
        self.early_stop = False
        self.best_model_state = None

    def __call__(self, val_loss, model):
        """
        Checks if the validation loss is diverging and updates the state accordingly.

        Args:
            val_loss (float): Current epoch's validation loss.
            model (torch.nn.Module): The model being trained.
        """
        if self.best_loss is None or val_loss < self.best_loss:
            # Improvement detected
            self.best_loss = val_loss
            self.best_model_state = model.state_dict()
            self.counter = 0
        else:
            # Validation loss increased
            self.counter += 1
            if self.counter >= self.patience:
                print(f"Divergence detected. Stopping training after {self.counter} epochs.")
                self.early_stop = True

    def load_best_model(self, model):
        """
        Restores the model to the state with the lowest validation loss.

        Args:
            model (torch.nn.Module): The model to restore.
        """
        model.load_state_dict(self.best_model_state)




In [5]:
import torch
import torch.nn as nn

debug_mode_flag = False
# Set the device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#set random seed for reproducibility
torch.manual_seed(42)
debug_mode_flag = False
import torch
import torch.nn as nn

class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout_rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, embed_dim)
        )
        self.layernorm1 = nn.LayerNorm(embed_dim)
        self.layernorm2 = nn.LayerNorm(embed_dim)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.dropout2 = nn.Dropout(dropout_rate)

    def forward(self, x):
        attn_output, _ = self.att(x, x, x)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(x + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)

class TransformerModel(nn.Module):
    def __init__(self, input_shape, num_classes, embed_dim=64, num_heads=2, ff_dim=64, num_transformer_blocks=2,dropout_rate=0.1):
        super(TransformerModel, self).__init__()
        self.reshape = nn.Flatten(start_dim=2)  # Reshaping as in TensorFlow's Reshape
        self.embedding = nn.Linear(input_shape[1] * input_shape[2], embed_dim)
        self.transformer_blocks = nn.ModuleList(
            [TransformerBlock(embed_dim, num_heads, ff_dim,dropout_rate=dropout_rate) for _ in range(num_transformer_blocks)]
        )
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        x = self.reshape(x)
        x = self.embedding(x)
        x = x.permute(1, 0, 2)  # PyTorch uses (seq_len, batch, embed_dim) format for transformers
        for transformer_block in self.transformer_blocks:
            x = transformer_block(x)
        x = x.permute(1, 2, 0)  # Back to (batch, embed_dim, seq_len)
        x = self.global_avg_pool(x).squeeze(-1)
        x = self.dropout(x)
        return self.classifier(x)


In [6]:
model  = TransformerModel(input_shape=(224,224,3),num_classes=3) # declare model here
randomdata = torch.randn((1,224,224,3))
output = model(randomdata)
output.shape

torch.Size([1, 3])

In [7]:
torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# pca = sklearn.decomposition.PCA(3)
epochs = 100


fold_indices = np.arange(5)
fold_indices = np.random.permutation(fold_indices)
val_fold_indices = np.roll(fold_indices, 1)

input_shape = (3,224,224)
num_classes = 3


In [8]:
import optuna
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import balanced_accuracy_score

# import warnings
# warnings.filterwarnings("ignore", category=UserWarning, message=".*step is already reported.*")

np.random.seed(42)  # Set random seed for reproducibility
torch.manual_seed(42)  # Set random seed for reproducibility



def objective(trial):
    # Hyperparameter suggestions
    #embed_dim=64, num_heads=2, ff_dim=64, num_transformer_blocks=2
    ff_dim = trial.suggest_categorical("ff_dim", [32, 64, 128, 256,512])
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.3)
    embed_dim = trial.suggest_categorical("embed_dim", [32, 64, 128, 256,512])
    learning_rate = trial.suggest_float("learning_rate", 1e-7, 1e-1, log=True)
    optimizer_name = "Adam"
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [10, 16, 32])
    num_heads = trial.suggest_categorical("num_heads", [1, 2, 4, 8, 16])
    num_transformer_blocks = trial.suggest_categorical("num_transformer_blocks", [1, 2, 4,8,16,32])
    label_smoothing = 0.3

    factor = 1
    

    print(f"Hyperparameters: num_heads={num_heads}, num_transformer_blocks={num_transformer_blocks}, "
          f"learning_rate={learning_rate}, optimizer={optimizer_name}, weight_decay={weight_decay}, batch_size={batch_size},factor={factor}")
    
    fold_accuracies = []

    for test_fold_idx in range(5):
        test_fold = fold_indices[test_fold_idx]
        remaining_folds = [fold_indices[i] for i in range(5) if i != test_fold_idx]
        val_fold_idx = test_fold_idx % 4
        val_fold = remaining_folds[val_fold_idx]
        train_folds = [fold for fold in remaining_folds if fold != val_fold]

        train_data = np.concatenate([eeg_folds[j] for j in train_folds]).transpose(0, 3, 1, 2)
        train_labels = np.concatenate([labels_folds[j] for j in train_folds])

        val_data = eeg_folds[val_fold].transpose(0, 3, 1, 2)
        val_labels = labels_folds[val_fold]

        test_data = eeg_folds[test_fold].transpose(0, 3, 1, 2)
        test_labels = labels_folds[test_fold]

        balanced_train_data, balanced_train_labels = data_balancer(train_data, train_labels, factor=factor)

        train_dataset = TensorDataset(torch.tensor(balanced_train_data, dtype=torch.float32), 
                                      torch.tensor(balanced_train_labels, dtype=torch.long))
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

        val_dataset = TensorDataset(torch.tensor(val_data, dtype=torch.float32), 
                                    torch.tensor(val_labels, dtype=torch.long))
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

        test_dataset = TensorDataset(torch.tensor(test_data, dtype=torch.float32), 
                                     torch.tensor(test_labels, dtype=torch.long))
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = TransformerModel(input_shape=input_shape, num_classes=num_classes, embed_dim=embed_dim, num_heads=num_heads, ff_dim=ff_dim,dropout_rate=dropout_rate, num_transformer_blocks=num_transformer_blocks).to(device)

        criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)
        optimizer_cls = {"Adam": optim.Adam, "AdamW": optim.AdamW, "SGD": optim.SGD}
        optimizer = optimizer_cls[optimizer_name](model.parameters(), lr=learning_rate, weight_decay=weight_decay)

        early_stopping = EarlyStopping(patience=10)

        epochs = 30
        for epoch in range(epochs):
            model.train()
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for val_inputs, val_labels in val_loader:
                    val_inputs, val_labels = val_inputs.to(device, non_blocking=True), val_labels.to(device, non_blocking=True)
                    val_outputs = model(val_inputs)
                    loss = criterion(val_outputs, val_labels)
                    val_loss += loss.item()

            val_loss /= len(val_loader)

            early_stopping(val_loss, model)
            if early_stopping.early_stop:
                print(f"Early stopping at epoch {epoch}")
                break

        early_stopping.load_best_model(model)

        model.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        fold_acc = balanced_accuracy_score(all_labels, all_preds)
        fold_accuracies.append(fold_acc)
        print(f"Trial {trial.number}, Fold {test_fold_idx+1}: Test Accuracy = {fold_acc:.4f}")

        del model
        torch.cuda.empty_cache()

    mean_accuracy = np.mean(fold_accuracies)
    print(f"Trial {trial.number}: Mean Accuracy = {mean_accuracy:.4f}, Fold Accuracies = {fold_accuracies}")
    
    trial.set_user_attr("fold_accuracies", fold_accuracies)
    trial.report(mean_accuracy, step=0)  # Single report after all folds

    if trial.should_prune():
        raise optuna.TrialPruned()

    
    return mean_accuracy

# Start Optuna Study
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(),
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=5, n_min_trials=5, interval_steps=1),
)

study.optimize(objective, n_trials=200, n_jobs=2)

# Best result
print("Best hyperparameters:", study.best_params)


c:\Users\Gabriel\anaconda3\envs\cudaenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-05-08 10:43:49,270] A new study created in memory with name: no-name-a0dbfd8a-de7e-4fb5-8258-5dce8477cfbc


Hyperparameters: num_heads=2, num_transformer_blocks=4, learning_rate=3.398124982496925e-06, optimizer=Adam, weight_decay=1.0773829757210749e-05, batch_size=10,factor=1
Hyperparameters: num_heads=4, num_transformer_blocks=2, learning_rate=6.572573477289355e-07, optimizer=Adam, weight_decay=6.043448462641347e-06, batch_size=32,factor=1
Divergence detected. Stopping training after 10 epochs.
Early stopping at epoch 10
Trial 0, Fold 1: Test Accuracy = 0.4055
Divergence detected. Stopping training after 10 epochs.
Early stopping at epoch 11
Trial 1, Fold 1: Test Accuracy = 0.5137
Divergence detected. Stopping training after 10 epochs.
Early stopping at epoch 13
Divergence detected. Stopping training after 10 epochs.
Early stopping at epoch 14
Trial 1, Fold 2: Test Accuracy = 0.3832
Trial 0, Fold 2: Test Accuracy = 0.3707
Divergence detected. Stopping training after 10 epochs.
Early stopping at epoch 10
Trial 1, Fold 3: Test Accuracy = 0.4898
Trial 0, Fold 3: Test Accuracy = 0.5248
Divergen

[I 2025-05-08 10:49:09,543] Trial 0 finished with value: 0.4158495582179203 and parameters: {'ff_dim': 256, 'dropout_rate': 0.2274490276744359, 'embed_dim': 128, 'learning_rate': 3.398124982496925e-06, 'weight_decay': 1.0773829757210749e-05, 'batch_size': 10, 'num_heads': 2, 'num_transformer_blocks': 4}. Best is trial 0 with value: 0.4158495582179203.


Trial 0, Fold 5: Test Accuracy = 0.3474
Trial 0: Mean Accuracy = 0.4158, Fold Accuracies = [np.float64(0.4054834054834055), np.float64(0.37066462649089765), np.float64(0.5247886473429951), np.float64(0.43087352462352463), np.float64(0.3474375871487785)]
Hyperparameters: num_heads=16, num_transformer_blocks=8, learning_rate=0.08298964825265703, optimizer=Adam, weight_decay=4.9773202290767266e-05, batch_size=32,factor=1


[I 2025-05-08 10:49:30,169] Trial 1 finished with value: 0.4422859443993431 and parameters: {'ff_dim': 128, 'dropout_rate': 0.2574972504954214, 'embed_dim': 512, 'learning_rate': 6.572573477289355e-07, 'weight_decay': 6.043448462641347e-06, 'batch_size': 32, 'num_heads': 4, 'num_transformer_blocks': 2}. Best is trial 1 with value: 0.4422859443993431.


...

Trial 197, Fold 5: Test Accuracy = 0.3481
Trial 197: Mean Accuracy = 0.4588, Fold Accuracies = [np.float64(0.5013528138528138), np.float64(0.4891713747645951), np.float64(0.5408212560386473), np.float64(0.4144408831908832), np.float64(0.34811376616430767)]
Hyperparameters: num_heads=4, num_transformer_blocks=32, learning_rate=6.455848583897662e-05, optimizer=Adam, weight_decay=4.4233916879810345e-05, batch_size=16,factor=1
Divergence detected. Stopping training after 10 epochs.
Early stopping at epoch 22


[I 2025-05-08 21:57:51,513] Trial 198 finished with value: 0.4947411457503975 and parameters: {'ff_dim': 32, 'dropout_rate': 0.11542281673201528, 'embed_dim': 256, 'learning_rate': 5.726209095539437e-05, 'weight_decay': 4.566099590369729e-05, 'batch_size': 16, 'num_heads': 2, 'num_transformer_blocks': 4}. Best is trial 198 with value: 0.4947411457503975.


Trial 198, Fold 5: Test Accuracy = 0.3863
Trial 198: Mean Accuracy = 0.4947, Fold Accuracies = [np.float64(0.57260101010101), np.float64(0.444634076166562), np.float64(0.5600241545893719), np.float64(0.5101114163614163), np.float64(0.3863350715336275)]
Trial 199, Fold 1: Test Accuracy = 0.4079
Divergence detected. Stopping training after 10 epochs.
Early stopping at epoch 28
Trial 199, Fold 2: Test Accuracy = 0.3319
Divergence detected. Stopping training after 10 epochs.
Early stopping at epoch 27
Trial 199, Fold 3: Test Accuracy = 0.4741
Divergence detected. Stopping training after 10 epochs.
Early stopping at epoch 17
Trial 199, Fold 4: Test Accuracy = 0.4393
Divergence detected. Stopping training after 10 epochs.
Early stopping at epoch 15


[I 2025-05-08 22:05:03,030] Trial 199 finished with value: 0.40790475842890583 and parameters: {'ff_dim': 32, 'dropout_rate': 0.10041588106079656, 'embed_dim': 256, 'learning_rate': 6.455848583897662e-05, 'weight_decay': 4.4233916879810345e-05, 'batch_size': 16, 'num_heads': 4, 'num_transformer_blocks': 32}. Best is trial 198 with value: 0.4947411457503975.


Trial 199, Fold 5: Test Accuracy = 0.3863
Trial 199: Mean Accuracy = 0.4079, Fold Accuracies = [np.float64(0.4079184704184704), np.float64(0.3319209039548023), np.float64(0.4740942028985507), np.float64(0.4392933455433455), np.float64(0.38629686932936025)]
Best hyperparameters: {'ff_dim': 32, 'dropout_rate': 0.11542281673201528, 'embed_dim': 256, 'learning_rate': 5.726209095539437e-05, 'weight_decay': 4.566099590369729e-05, 'batch_size': 16, 'num_heads': 2, 'num_transformer_blocks': 4}
